# OpenGGCM_GM


The kamodofied OpenGGCM_GM reader requires `netCDF4` (and numpy, scipy, and pandas, but those are required by kamodo)

    pip install netCDF4

In [1]:
#import block
%load_ext autoreload
%autoreload 2
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected = True)
import numpy as np
from kamodo.readers.openggcm_gm_4Dcdf_xarray import MODEL, model_varnames
from kamodo.readers.reader_kplots import plot2D
from time import perf_counter
t0=perf_counter()
reader = MODEL()
print(perf_counter()-t0, 's')

0.0008022000000003082 s


In [2]:
#print possible variable names for given model
print('Standardized variables:\n---------------------------')
for key in model_varnames.keys():
    print(model_varnames[key][0]+':', model_varnames[key][1:])

Standardized variables:
---------------------------
B_x: ['x component of magnetic field', 0, 'GSE', 'car', ['time', 'x', 'y', 'z'], 'nT']
B_y: ['y component of magnetic field', 1, 'GSE', 'car', ['time', 'x', 'y', 'z'], 'nT']
B_z: ['z component of magnetic field', 2, 'GSE', 'car', ['time', 'x', 'y', 'z'], 'nT']
B1_x: ['x component of magnetic field (on grid cell faces)', 3, 'GSE', 'car', ['time', 'x', 'x', 'x'], 'nT']
B1_y: ['y component of magnetic field (on grid cell faces)', 4, 'GSE', 'car', ['time', 'y', 'y', 'y'], 'nT']
B1_z: ['z component of magnetic field (on grid cell faces)', 5, 'GSE', 'car', ['time', 'z', 'z', 'z'], 'nT']
E_x: ['x component of electric field (on grid cell edges)', 6, 'GSE', 'car', ['time', 'x', 'x', 'x'], 'mV/m']
E_y: ['y component of electric field (on grid cell edges)', 7, 'GSE', 'car', ['time', 'y', 'y', 'y'], 'mV/m']
E_z: ['z component of electric field (on grid cell edges)', 8, 'GSE', 'car', ['time', 'z', 'z', 'z'], 'mV/m']
V_x: ['x component of plasma v

In [3]:
#set filename, run_name, desired variable list, and whether gridded interpolators are desired
run_name =  "Yihua_Zheng_090721_1"
var_list, gridded_int = ['N_plasma','B_x','B1_x','E_x','V_x','eta'], True  #
var_3D = ''  #for testing of expected plot error in last cell, should be a 3D variable name (if possible)

In [4]:
#set variables to be plotted
if gridded_int:  #set list of variables to check plots, list should include at least one 3D and at least one 4D variable
    plot_list = list(np.ravel([[var, var+'_ijk'] for var in var_list]))
else:
    plot_list = var_list
print(plot_list)

['N_plasma', 'N_plasma_ijk', 'B_x', 'B_x_ijk', 'B1_x', 'B1_x_ijk', 'E_x', 'E_x_ijk', 'V_x', 'V_x_ijk', 'eta', 'eta_ijk']


In [5]:
file_prefix = 'D:/OpenGGCM_GM/Data/Yihua_1/Yihua_Zheng_090721_1.3df_2015-10-16_12'
tic = perf_counter()
kamodo_object = reader(file_prefix, runname=run_name)  #load all variables
#kamodo_object = reader(file_prefix, variables_requested=['N_plasma'], runname=run_name, 
#                       gridded_int=True)  #load specified variables with gridded interpolators
print(perf_counter()-tic, 's for ', len(kamodo_object.variables.keys()), ' variables')

3.4202832 s for  24  variables


In [6]:
#show functionalized variables in kamodo object
kamodo_object

{B_x(xvec): <function MODEL.<locals>.MODEL.custom_interp.<locals>.interpolator at 0x000001E563538288>, B_x: <function MODEL.<locals>.MODEL.custom_interp.<locals>.interpolator at 0x000001E563538288>, B_x_ijk(time, x, y, z): <function MODEL.<locals>.MODEL.custom_interp.<locals>.gridded_interpolator at 0x000001E563538438>, B_x_ijk: <function MODEL.<locals>.MODEL.custom_interp.<locals>.gridded_interpolator at 0x000001E563538438>, B_y(xvec): <function MODEL.<locals>.MODEL.custom_interp.<locals>.interpolator at 0x000001E563538798>, B_y: <function MODEL.<locals>.MODEL.custom_interp.<locals>.interpolator at 0x000001E563538798>, B_y_ijk(time, x, y, z): <function MODEL.<locals>.MODEL.custom_interp.<locals>.gridded_interpolator at 0x000001E563538C18>, B_y_ijk: <function MODEL.<locals>.MODEL.custom_interp.<locals>.gridded_interpolator at 0x000001E563538C18>, B_z(xvec): <function MODEL.<locals>.MODEL.custom_interp.<locals>.interpolator at 0x000001E5623C2438>, B_z: <function MODEL.<locals>.MODEL.cus

In [7]:
kamodo_object._time.min(), kamodo_object._time.max()

(12.000021, 13.00001)

In [8]:
kamodo_object._x.min(), kamodo_object._x.max()

(-350.0119, 33.01)

In [9]:
kamodo_object._y.min(), kamodo_object._y.max()

(-96.01, 96.01)

In [10]:
kamodo_object._z.min(), kamodo_object._z.max()

(-96.01, 96.01)

In [11]:
#show variable coordinate dependencies (especially for non-gridded variables)
for key in kamodo_object.variables.keys(): print(key, kamodo_object.variables[key]['xvec'])

B_x {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
B_x_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
B_y {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
B_y_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
B_z {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
B_z_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
V_x {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
V_x_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
V_y {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
V_y_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
V_z {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
V_z_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
J_x {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
J_x_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
J_y {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
J_y_ijk {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
J_z {'time': 'hr', 'x': 'R_E', 'y': 'R_E', 'z': 'R_E'}
J_z_ijk {'time': 'hr', 'x': 'R_E'

In [12]:
kamodo_object.N_plasma.data

<xarray.DataArray 'rr' (time: 61, x: 656, y: 190, z: 190)>
dask.array<open_dataset-f10c69aac6d38ea0bbd6ef18e787ac85rr, shape=(61, 656, 190, 190), dtype=float32, chunksize=(61, 100, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) float32 12.0 12.02 12.03 12.05 ... 12.95 12.97 12.98 13.0
  * x        (x) float32 -350.0 -348.4 -346.8 -345.2 ... 32.72 32.82 32.91 33.01
  * y        (y) float32 -96.01 -93.07 -90.2 -87.39 ... 87.39 90.2 93.07 96.01
  * z        (z) float32 -96.01 -93.07 -90.2 -87.39 ... 87.39 90.2 93.07 96.01
Attributes:
    units:    1/cm**3

In [13]:
kamodo_object.N_plasma.data.interp(time=12.8,x=5.,y=0.,z=0.).values

array(2.60609573)

In [14]:
kamodo_object.N_plasma([[12.8,5.,0.,0.]])

Took 0.019 s for 1 positions.


array([2.60609573])

In [15]:
kamodo_object.N_plasma_ijk(time=12.8,x=5.,y=0.,z=0.)

array(2.60609573)

In [16]:
#show documentation for 2D plotting function
help(plot2D)

Help on function plot2D in module kamodo.readers.reader_kplots:

plot2D(kamodo_object, varname, plottype, t, lon, lat, h=-1)
    Use Kamodo's native plotting to generate 2D plot.
    t, lon, lat, and h also double as t, x, y, and z for cartesian inputs.
    Possible plot types are LonLat, LatH, LonH, TimeLat, TimeLon, and TimeH for
        spherical coordinates; and TimeX, TimeY, TimeX, XY, XZ, and YZ for
        cartesian coordinates.
    If a LonLat plot is requested, then the function expects a single value
        (integer, float, float32, or float64) for t and h (if h is given).
        In this case, lon and lat should be 1D arrays or flat lists. 
    If the variable depends on height, then a value or array should be given for h.



In [17]:
#generate each possible type of 2D plot for each variable in plot_list
xval, xarr = 0., np.linspace(-15.,15.,20)
yval, yarr = 0., np.linspace(-15.,15.,20)
zval, zarr = 2., np.linspace(-20.,20.,20)
timeval, timearr = 12.5, np.linspace(12.,13.,20)
for plot_var in plot_list:
    if plot_var not in kamodo_object.variables: 
        print(plot_var)
        continue
    print(kamodo_object.variables[plot_var]['xvec'].keys())
    print('---------------------------------------------------------------')
    print(f'--------------  Plots for {plot_var}  -----------------------')
    print('---------------------------------------------------------------')
    if len(list(kamodo_object.variables[plot_var]['xvec'].keys()))==4:  #if height depedent
        #make 2D plots for variable
        iplot(plot2D(kamodo_object, plot_var,'XY',timeval,xarr,yarr,zval))
        iplot(plot2D(kamodo_object, plot_var,'TimeY',timearr,xval,yarr,zval))
        iplot(plot2D(kamodo_object, plot_var,'TimeX',timearr,xarr,yval,zval))
        iplot(plot2D(kamodo_object, plot_var,'XZ',timeval,xarr,yval,zarr))
        iplot(plot2D(kamodo_object, plot_var,'YZ',timeval,xval,yarr,zarr))
        iplot(plot2D(kamodo_object, plot_var,'TimeZ',timearr,xval,yval,zarr))

dict_keys(['time', 'x', 'y', 'z'])
---------------------------------------------------------------
--------------  Plots for N_plasma  -----------------------
---------------------------------------------------------------
N_plasma
Time slice at 12.500 hrs. Z slice at 2.0000000 R_E.
Took 7.462 s for 400 positions.


N_plasma
X slice at 0.000 R_E. Z slice at 2.0000000 R_E.
Took 7.054 s for 400 positions.


N_plasma
Y slice at 0.000 R_E. Z slice at 2.0000000 R_E.
Took 6.858 s for 400 positions.


N_plasma
Time slice at 12.500 hrs. Y slice at 0.000 R_E. 
Took 7.441 s for 400 positions.


N_plasma
Time slice at 12.500 hrs. X slice at 0.000 R_E. 
Took 7.560 s for 400 positions.


N_plasma
X slice at 0.000 R_E. Y slice at 0.000 R_E. 
Took 6.850 s for 400 positions.


dict_keys(['time', 'x', 'y', 'z'])
---------------------------------------------------------------
--------------  Plots for N_plasma_ijk  -----------------------
---------------------------------------------------------------
N_plasma_ijk
Time slice at 12.500 hrs. Z slice at 2.0000000 R_E.
not supported: out_dim (1, 'N', 'N', 'N'), arg_dims [('N',), ('N',)]


KeyError: (1, 'N', 'N', 'N')

In [18]:
#Show proper errors when user tries to plot a time+2D variable against height
if var_3D in kamodo_object.variables.keys():
    try:
        iplot(plot2D(kamodo_object, var_3D,'LonH',timeval,lonarr,latval,height_arr))
    except AttributeError as e:
        print(e)
    try:
        iplot(plot2D(kamodo_object, var_3D,'LatH',timeval,lonval,latarr,height_arr))
    except AttributeError as e:
        print(e)   
    try:
        iplot(plot2D(kamodo_object, var_3D,'TimeH',timearr,lonval,latval,height_arr))
    except AttributeError as e:
        print(e)   
else:
    print('No 3D variable given.')

No 3D variable given.


In [19]:
kamodo_object.datetimes

['2015-10-16 12:00:00.075531', '2015-10-16 13:00:00.034332']

In [20]:
print('Inner boundary radius:', kamodo_object.near_Earth_boundary_radius, 
                  kamodo_object.near_Earth_boundary_radius_unit)  #should be 2.5 R_E

Inner boundary radius: 2.5 R_E


In [ ]:
kamodo_object.filetimes